In [1]:
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [17]:
beers = pd.read_json("data/beers.json", orient='columns', lines=True)
beers.head()

,abv,beer_id,brewery,brewery_id,name,num_reviews,pDev,rAvg,style,style_id,timestamp
0,8.0,77364,Murree Brewery Co. Ltd.,16097,Murree's Millennium,0,0.0,3.8,Euro Strong Lager,43,2013-07-01 17:46:33
1,5.2,92762,"Nigerian Breweries, Ltd.",8267,Gulder Lager (Nigeria),1,4.6,2.6,American Adjunct Lager,38,2013-07-01 17:46:33
2,5.3,14952,"Nigerian Breweries, Ltd.",8267,Star Lager,5,15.2,3.0,American Adjunct Lager,38,2013-07-01 17:46:33
3,3.5,26316,Industrial Cervecera S.A. / Compañía Cervecera...,9048,Victoria Light,1,0.0,4.2,Light Lager,39,2013-07-01 17:46:33
4,9.0,87583,Bierbrouwerij Maallust,27084,Zware Jongen Tripel Bier,3,6.7,3.9,Tripel,58,2013-07-01 17:46:34


In [18]:
user_ratings = pd.read_json("data/my_untappd.json", orient='columns', lines=False)
user_ratings.head()

,beer_abv,beer_ibu,beer_name,beer_type,beer_url,brewery_city,brewery_country,brewery_name,brewery_state,brewery_url,...,flavor_profiles,purchase_venue,rating_score,serving_type,venue_city,venue_country,venue_lat,venue_lng,venue_name,venue_state
0,6.2,44,Long Hammer IPA,IPA - American,https://untappd.com/beer/6504,Seattle,United States,Redhook Brewery,WA,https://untappd.com/brewery/1051,...,,,3.5,,Singapore,Singapore,1.3,103.8,Brewerkz Restaurant & Microbrewery,
1,5.0,40,Single Hop Series: Lemondrop,Pale Ale - American,https://untappd.com/beer/1075979,Singapore,Singapore,Brewerkz,,https://untappd.com/brewery/2454,...,,,3,,None,None,nan,nan,None,None
2,5.5,10,Black Raspberry Ale,Fruit Beer,https://untappd.com/beer/606637,Singapore,Singapore,Brewerkz,,https://untappd.com/brewery/2454,...,,,,,None,None,nan,nan,None,None
3,4.5,20,Golden Ale (1997 - 2018),Golden Ale,https://untappd.com/beer/21704,Singapore,Singapore,Brewerkz,,https://untappd.com/brewery/2454,...,,,,,None,None,nan,nan,None,None
4,8.0,60,Black Pig,IPA - Black / Cascadian Dark Ale,https://untappd.com/beer/180128,Singapore,Singapore,Brewerkz,,https://untappd.com/brewery/2454,...,,,3.75,,None,None,nan,nan,None,None


In [14]:
beers.describe()

,abv,beer_id,brewery_id,num_reviews,pDev,rAvg,style_id
count,52839.0,68075.0,68075.0,68075.0,68075.0,68075.0,68075.0
mean,6.5,50669.2,9011.6,32.3,8.1,3.5,92.7
std,2.1,28056.1,9650.6,139.8,7.8,0.9,47.3
min,0.0,5.0,3.0,0.0,0.0,0.0,5.0
25%,5.0,27052.5,870.0,1.0,0.0,3.4,55.0
50%,6.0,52692.0,3925.0,2.0,7.6,3.7,93.0
75%,7.5,75122.5,16792.0,8.0,12.7,4.0,129.0
max,65.0,96000.0,32009.0,3303.0,91.7,5.0,175.0


In [16]:
user_ratings.describe()

,beer_abv,beer_ibu,venue_lat,venue_lng
count,1101.0,1101.0,665.0,665.0
mean,6.4,29.9,-3.5,107.6
std,2.0,42.8,17.6,30.7
min,0.0,0.0,-38.3,-73.7
25%,5.0,0.0,1.3,103.8
50%,6.0,21.0,1.3,103.8
75%,7.2,52.0,1.3,103.9
max,22.0,1000.0,45.5,145.0
